# Introduction
- Our team's project is to create coding to do data analysis.
- Our team members are 김동구, 김민재, 박찬근.
- The role of team members is to develop code and communicate with customer. And professor's role is a mentor and client for our team.



# Objective of project
- This objective of project is to analyze the received raw data and deliver it in the form of a documentation file.


# Structure of code
![flow_chart.png](./image/flow_chart.png)
- 위 그림은 이 프로그램의 전체적인 flow chart입니다. src module 7개와 run.py 이렇게 총 8개 module로 구성되어 있습니다.
- The figure above is the overall flow chart of this program. It consists of a total of 8 modules : 7 src modules and run.py.
    
- 프로그램은 run.py를 실행하게 되면 process.py에서 여러 module들이 연산하고 plot한 data들을 하나로 통합하여 저장합니다.
- When the program runs run.py, In process.py, several modules calculate. And plotted data are saved into integrated form. 

- 이제부터 각 module에 대해 설명하겠습니다.
- From now, we will explain each module.

##  1. run.py
  - 프로그램을 시동하는 역할을 하는 module입니다. data folder에 분석할 data를 넣고 run.py를 실행하게 되면 프로그램의 중심이 되는 process.py를 import해 프로그램을 실행하게 됩니다.
  - run.py is the module responsible for running the program. When you put the data to be analyzed in the data folder and run run.py, the program is operated by importing process.py.

##  2. process.py
  - 프로그램의 중심이 되는 module입니다. data folder에 있는 raw data xml 파일들을 파싱하고 plot module과 fitting module에 data를 전달하고 이 module들이 plot한 figure를 모아 png 파일로 저장하고 이외의 정보들을 csv 파일로 저장하는 역할을 합니다.
  - process.py is the central module of the program. It parses raw data xml files in the data folder and delivers the data to the plot module and fitting module, collects the plotted figure, and saves it as a png file and saves other information as a csv file.
![process.png](./image/process.png)
```python
    plt.figure(figsize = (18,12))
    plt.subplot(234)
    IVvoltage = tree.findall(
        'ElectroOpticalMeasurements/ModulatorSite/Modulator/PortCombo/IVMeasurement/Voltage')
    IVcurrent = tree.findall(
        'ElectroOpticalMeasurements/ModulatorSite/Modulator/PortCombo/IVMeasurement/Current')
    IVplot(IVvoltage,IVcurrent)
    IVfitting(IVvoltage,IVcurrent)
```
    
    - raw data를 모듈에 주고 plot된 figure를 받아오는 코드
    - Code that gives raw data to the module and receives the plotted figure.

## 3. data_set.py

```python
    def data_set(data,ab=0):
        value = []
        for i in range(0,len(data)):
            value1 = data[i].text.split(',')
            value1 = list(map(float,value1))
            if ab == 0:
                value.append(value1)
            elif ab == 1:
                value2 = list(map(abs,value1))
                value.append(value2)
        return value
```
  - 각 raw data들의 str 형식으로 나열되어 있는 값들을 연산이 가능한 float으로 구성된 리스트로 바꿔주는 module.
  -Module that converts the values listed in str format of each raw data into a list of float that can be calculated on.
  
## 4. R_square.py
```python
def R_square(y_value,fit_value):
    y_value_mean = np.mean(np.ravel(y_value))
    Tss = np.sum((np.ravel(y_value)-y_value_mean)**2)
    Rss = np.sum((np.ravel(y_value)-np.ravel(fit_value))**2)
    results = 1 - (Rss/Tss)
    return results
```
  - fitting 함수의 Rsq값을 계산해주는 module.
  - Module that calculates Rsq value of the fitting function.

## 5. poly_fitting.py
```python
def polyfitting_p1d1(number,x,y):
    p_list = []
    for i in range(13,number):
        p_list.append(np.polyfit(x,y,i))
    p1d_list = []
    for i in range(13,number):
        p1d_list.append(np.poly1d(p_list[i-13]))
    return p1d_list
```
  - numpy의 polyfit을 이용하여 입력받은 raw data를 다항함수로 fitting 해주는 module. for문의 시작 숫자부터 number까지의 차수로 피팅하고 각각을 리스트에 저장한다.
  - Module that fitting the inputed raw data with a polynomial function using numpy's polyfit. Fitting from the starting number of for loop to the order of number and store each in a list.

## 6. ILplot.py
```python
def IL_raw_plot(x,y):
    IL_x = data_set(x)
    IL_y = data_set(y)
    for i in range(0,len(x)):
        plt.scatter(IL_x[i],IL_y[i],label = DC_li[i])
    plt.ylabel('Measured transmission[dB]')
    plt.xlabel('Wavelength[nm]')
    plt.title('Transmission spectra - as measured')
    plt.legend(ncol=4)

def IL_fitting_ref(x,y,order=6):
    IL_x = data_set(x)
    IL_y = data_set(y)
    rs_list = R_square.find_R_square(order, IL_x[6], IL_y[6])
    fit_data = polyfitting_po(order, IL_x[6], IL_y[6])
    for i in range(4,order):
        plt.plot(IL_x[6],fit_data[i-4],label='{}{} {} {}'.format(i,'$^{th}$','$R^{2}$ =',round(rs_list[i-4],5)))
    plt.legend(ncol=2, loc=8)
    plt.ylabel('Measured transmission[dB]')
    plt.xlabel('Wavelength[nm]')
    plt.title('Transmission spectra - processed ref')

def IL_processed_plot(x,y,order=6):
    IL_x = data_set(x)
    IL_y = data_set(y)
    ind_IL = R_square.find_best_fit_ind(order, IL_x[6], IL_y[6])
    fit_data = polyfitting_po(order,IL_x[6],IL_y[6])
    for i in range(0, len(x)):
        x = IL_x[i]
        y = []
        try:
            for k in range(0,len(IL_y[i])):
                y.append(float(IL_y[i][k])-fit_data[ind_IL][k])
        except:
            x = IL_x[i][0:len(fit_data[ind_IL])]
            y = y[0:len(fit_data[ind_IL])]
        plt.plot(x, y, label=DC_li[i])
    plt.legend(ncol=4)
    plt.ylabel('Measured transmission[dB]')
    plt.xlabel('Wavelength[nm]')
    plt.title('Transmission spectra - as measured & processed for {}{} fit ref'.format(ind_IL+4, '$^{th}$'))
```
  - Intensity와 Wavelength의 raw data plot, reference fitting function plot, fitting 함수로 raw data를 가공한 plot까지 총 3개의 그래프를 그리는 module. 정의된 함수 이름과 역할이 같다.
  - Module that draws a total of 3 graphs, including Intestiy and Wavelength raw data plot, reference fitting function plot, and plot processed with raw data by fitting function. The defined function name and role are same.

## 7. exp_fitting.py
```python
def IVfitting(x,y,number=0):
    global R
    def expfuc(x, a, b, c):
        return b * (np.exp(a * x) - 1) * c + IV_fit_value[i](x)
    IV_x = data_set(x)
    IV_y = data_set(y,1)
    IV_fit_value = polyfitting_p1d1(15,IV_x[number][:10],IV_y[number][:10])
    emodel = Model(expfuc)
    fit_result = []
    fit = []
    for i in range(0,len(IV_fit_value)):
        result1 = emodel.fit(IV_y[number],x = IV_x[number], b = 1, a = 1, c = 1)
        R = R_square(IV_y[number],result1.best_fit)
        fit_result.append(R)
        fit.append(result1.best_fit)
    rs_min = []
    for i in range(0,len(fit_result)):
        rs_min.append(abs(fit_result[i]-1))
    rs_min2 = min(rs_min)
    IV_ind = rs_min.index(rs_min2)
    R = fit_result[IV_ind]

    global IV_dic
    plt.plot(IV_x[number],fit[IV_ind],'r-',label = '{} {} {}'.format('processed',IV_ind+13,'th'))
    IV_dic = {y:x for x,y in zip(fit[IV_ind],IV_x[number])}
    plt.text(1.0,IV_dic[1.0],IV_dic[1.0],fontsize = 8,horizontalalignment = 'right')
    plt.text(-1.0,IV_dic[-1.0],IV_dic[-1.0],fontsize = 8)
    plt.xlabel('Voltage[V]')
    plt.ylabel('Current[A]')
    plt.title('IV-analysis raw & processed')
    plt.yscale("log")
    plt.legend(loc = 'best')
```
  - IV fitting의 경우 shockley equation 만으로 fitting하기 힘들기 때문에 다항함수 항을 추가하여 fitting한다. 앞서 설명한 poly_fitting.py에서 다항함수 피팅을 먼저 한 후 하나씩 exp 함수와 더해 각각 fitting하고 Rsq를 비교해 가장 정확한 fitting함수를 찾고 plot까지 진행하는 module.
  - In the case of IV fitting, It is difficult to fitting only with the shockley equation, so that polynomial function term is added for fitting. 
  - Module in poly_fittinf.py, polynomial function fitting is first perfomed, then each is added with the exp function to fitting each and compare Rsq to find the most accurate fitting function and proceed to the plot. 
    
  
## 8. IVplot.py
```python
def IVplot(x,y,number=0):
    IV_x = data_set(x)
    IV_y = data_set(y, 1)
    plt.scatter(IV_x[number], IV_y[number], label='{}'.format('Raw IV'))
    plt.xlabel('Voltage[V]')
    plt.ylabel('Current[A]')
    plt.title('IV-analysis raw & processed')
    plt.yscale("log")
```
  - IV의 raw data를 plot하는 module.
  - Module that plot raw data of IV

# Example of analysis result
  - sample image of analysis result
![example_figure.png](./image/example_figure.png)

  - a table in csv file


In [8]:
!pip install pandas
import pandas as pd
df = pd.read_csv(r'C:\Users\bluepc2020\PycharmProjects\project\result\result.csv')
print(df)

    Unnamed: 0      Lot Wafer      Mask  TestSite                  Name  \
0          NaN  P184640   D07  GORILLA5  DCM_LMZC  MZMCTE_LULAB_450_500   
1          NaN  P184640   D07  GORILLA5  DCM_LMZC  MZMCTE_LULAB_450_500   
2          NaN  P184640   D07  GORILLA5  DCM_LMZC  MZMCTE_LULAB_450_500   
3          NaN  P184640   D07  GORILLA5  DCM_LMZC  MZMCTE_LULAB_450_500   
4          NaN  P184640   D07  GORILLA5  DCM_LMZC  MZMCTE_LULAB_450_500   
..         ...      ...   ...       ...       ...                   ...   
93         NaN  P184640   D24  GORILLA5  DCM_LMZO  MZMOTE_LULAB_380_500   
94         NaN  P184640   D24  GORILLA5  DCM_LMZO  MZMOTE_LULAB_380_500   
95         NaN  P184640   D24  GORILLA5  DCM_LMZO  MZMOTE_LULAB_380_500   
96         NaN  P184640   D24  GORILLA5  DCM_LMZO  MZMOTE_LULAB_380_500   
97         NaN  P184640   D24  GORILLA5  DCM_LMZO  MZMOTE_LULAB_380_500   

               Date  Operator  Row  Column  error_flag error_description  \
0   20190715_191703    

# Result of wafer-to-wafer using csv file

In [ ]:
!pip install matplotlib
import pandas as pd

df = pd.read_scv(C:\Users\bluepc2020\PycharmProjects\project\result\result.csv)
